## Advanced RAG Demo: Conceptual Graph RAG

This notebook provides a *conceptual demonstration* of how one might begin to incorporate graph-based elements into a RAG system. A full Graph RAG system often involves dedicated graph databases (like Neo4j) and more complex graph construction and traversal algorithms. Here, we'll simplify by:

1.  Loading and chunking a PDF as usual.
2.  Using an LLM (Groq) to perform a basic form of **entity and simple relationship extraction** from text chunks. This is a key step in building a knowledge graph.
3.  Storing the original text chunks in a standard vector store (FAISS).
4.  For a given query, we will:
    a.  Retrieve relevant text chunks using vector search.
    b.  For these top chunks, use our (simulated) extracted entity/relationship information.
    c.  Augment the prompt to the final generator LLM with both the raw text of the retrieved chunks AND the extracted structured (graph-like) information.

**Goal:** To show how adding structured relational information, even if simply extracted, could potentially help the LLM understand context better and answer more nuanced questions.

**Note:** This is a simplified approach. True Graph RAG would involve building and querying an actual graph structure.

### 1. Setup: Install Libraries and Import Modules

In [ ]:
!pip install -q langchain langchain-groq langchain-community pypdf faiss-cpu pypdf sentence-transformers

In [ ]:
import os
import getpass
import json # For handling LLM output that might be structured
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_core.documents import Document # To handle chunk metadata

### 2. Configure Groq API Key

In [ ]:
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

In [ ]:
os.makedirs("pdfs", exist_ok=True)

# Step 3: Download the PDF using requests
import requests

url = "https://cs229.stanford.edu/main_notes.pdf"
pdf_path = "pdfs/main_notes.pdf"

response = requests.get(url)
with open(pdf_path, "wb") as f:
    f.write(response.content)

print(f"PDF downloaded to: {pdf_path}")

### 3. Prepare PDF Document & Vector Store (Abbreviated)

In [ ]:
raw_chunks = [] # Will store Document objects
vector_store = None

if os.path.exists(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    raw_chunks = text_splitter.split_documents(documents)
    print(f"Split into {len(raw_chunks)} chunks.")

    if raw_chunks:
        embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
        embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
        print("Creating FAISS vector store...")
        vector_store = FAISS.from_documents(raw_chunks, embeddings)
        print("FAISS vector store created.")
else:
    print("PDF not found, skipping processing.")

### 4. LLM for Entity & Relationship Extraction (Simplified)

We'll define a prompt to ask the LLM to extract key concepts (entities) and simple relationships between them from a given text chunk. We'll ask for output in a structured format (like JSON) for easier parsing, though LLMs can be finicky with strict JSON.

In [ ]:
extraction_llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.0) # Low temp for more factual extraction

extraction_prompt_template = """
From the following text, extract up to 5 key machine learning concepts (as entities) and any simple relationships between them (e.g., 'Concept A is a type of Concept B', 'Concept X uses Concept Y').
Present the output as a JSON object with two keys: "entities" (a list of strings) and "relationships" (a list of strings describing the relationships).
If no clear concepts or relationships are found, return empty lists.

Example:
Text: 'Support Vector Machines (SVMs) are a type of supervised learning algorithm. SVMs can use different kernels, like the RBF kernel, to find a hyperplane.'
Output:
```json
{
  "entities": ["Support Vector Machines", "supervised learning algorithm", "kernels", "RBF kernel", "hyperplane"],
  "relationships": ["Support Vector Machines are a type of supervised learning algorithm", "SVMs can use kernels", "RBF kernel is a type of kernel"]
}
```

Text to process:
---
{text_chunk}
---

Output (JSON):
"""
extraction_prompt = PromptTemplate.from_template(extraction_prompt_template)
extraction_chain = LLMChain(llm=extraction_llm, prompt=extraction_prompt)

def extract_graph_elements(text_chunk_content):
    try:
        response = extraction_chain.invoke({"text_chunk": text_chunk_content})
        # Try to parse the JSON output, cleaning it up if necessary
        # LLMs sometimes add ```json ... ``` around the actual JSON
        content = response['text']
        if content.startswith("```json"):
            content = content[7:]
        if content.endswith("```"):
            content = content[:-3]

        data = json.loads(content.strip())
        return data
    except Exception as e:
        # print(f"Error parsing JSON from LLM: {e}\nLLM Output: {response['text']}")
        return {"entities": [], "relationships": []} # Default on error

### 5. Augment Chunks with Extracted Graph-like Data (Simulated)

In a real system, you might store this in a graph DB or link it to your chunks. Here, we'll extract it on-the-fly for the top retrieved chunks for simplicity in the demo. For a larger scale, pre-processing and storing these extractions would be necessary.

In [ ]:
# Let's try extracting for a few sample chunks to see it work
if raw_chunks and len(raw_chunks) > 5:
    print("--- Example Extractions ---")
    for i, chunk_doc in enumerate(raw_chunks[0:2]): # Process first 2 chunks as example
        print(f"\nChunk {i+1} (first 100 chars): {chunk_doc.page_content[:100]}...")
        graph_data = extract_graph_elements(chunk_doc.page_content)
        print(f"Extracted Entities: {graph_data.get('entities')}")
        print(f"Extracted Relationships: {graph_data.get('relationships')}")
        # In a real system, you'd add this to the chunk's metadata
        # chunk_doc.metadata['graph_elements'] = graph_data
    print("-------------------------")

### 6. RAG with Augmented Context (Text + Extracted Graph Elements)

Now, when we retrieve chunks for a query, we'll also (conceptually) fetch their pre-extracted graph elements and include them in the prompt to the final generator LLM.

In [ ]:
generator_llm = ChatGroq(model_name="llama3-70b-8192", temperature=0.2)

graph_augmented_qa_template = """
You are a helpful AI assistant answering questions based on the provided context.
The context consists of text passages and some extracted key concepts and relationships from those passages.
Use all this information to formulate a comprehensive and accurate answer.
If the information is not sufficient, say so.

--- CONTEXT ---
{context_str}
--- END CONTEXT ---

Question: {query_str}
Helpful Answer:
"""
graph_qa_prompt = PromptTemplate.from_template(graph_augmented_qa_template)
graph_qa_chain = LLMChain(llm=generator_llm, prompt=graph_qa_prompt)

def answer_with_graph_context(query, vector_store_retriever, k_chunks=2):
    if not vector_store_retriever:
        return "Vector store not available."

    print(f"Retrieving top {k_chunks} chunks for query: '{query}'")
    retrieved_docs = vector_store_retriever.get_relevant_documents(query)

    if not retrieved_docs:
        return "No relevant documents found."

    augmented_context_parts = []
    for i, doc in enumerate(retrieved_docs):
        text_part = f"Passage {i+1}:\n{doc.page_content}"
        augmented_context_parts.append(text_part)

        # Simulate fetching/using pre-extracted graph data for this chunk
        # For this demo, we'll extract it on the fly. In production, this would be pre-computed.
        print(f"Extracting graph elements for retrieved chunk {i+1}...")
        graph_elements = extract_graph_elements(doc.page_content)
        if graph_elements["entities"] or graph_elements["relationships"]:
            graph_part = f"Key Concepts/Relationships from Passage {i+1}:\n"
            if graph_elements["entities"]:
                graph_part += f"  Entities: {', '.join(graph_elements['entities'])}\n"
            if graph_elements["relationships"]:
                graph_part += f"  Relationships: {'; '.join(graph_elements['relationships'])}"
            augmented_context_parts.append(graph_part)

    full_context = "\n\n---\n\n".join(augmented_context_parts)
    # print(f"\n--- Augmented Context for LLM ---\n{full_context[:1500]}...\n---------------------------------")

    print("\nGenerating answer with augmented context...")
    response = graph_qa_chain.invoke({"context_str": full_context, "query_str": query})
    return response['text']

### 7. Ask a Question using Conceptual Graph-Augmented RAG

In [ ]:
if vector_store:
    retriever = vector_store.as_retriever(search_kwargs={'k': 2}) # Retrieve top 2 chunks

    # A query where relationships between concepts might be useful
    graph_query = "Explain how Gaussian Discriminant Analysis relates to logistic regression, and what assumptions GDA makes."
    print(f"\n--- Conceptual Graph RAG Query: {graph_query} ---")
    graph_answer = answer_with_graph_context(graph_query, retriever)
    print(f"\nConceptual Graph RAG Answer:\n{graph_answer}")

    # Another example
    graph_query_2 = "What are some types of kernels used in Support Vector Machines and how do they work?"
    print(f"\n--- Conceptual Graph RAG Query: {graph_query_2} ---")
    graph_answer_2 = answer_with_graph_context(graph_query_2, retriever, k_chunks=3)
    print(f"\nConceptual Graph RAG Answer:\n{graph_answer_2}")
else:
    print("Cannot run conceptual graph RAG as vector store is not available.")

### 8. Conclusion

This notebook offered a conceptual glimpse into Graph RAG. We simulated the extraction of entities and relationships from text chunks and showed how this structured information could be combined with raw text to form a richer context for the generator LLM.

**Key Takeaways:**
- Extracting structured data (like graph elements) from unstructured text can add significant value.
- Providing both textual context and related structured information can potentially lead to more nuanced and accurate LLM responses, especially for questions involving relationships between concepts.
- Full Graph RAG systems are more complex, involving robust graph construction, storage (e.g., in a graph database like Neo4j, Kuzu, or NebulaGraph), and sophisticated graph traversal/querying techniques integrated into the retrieval process.

This simplified demo highlights the potential direction. For a production system, you would invest in more robust entity/relationship extraction, potentially use a dedicated graph database, and design more sophisticated ways to combine graph query results with text retrieval.